<a href="https://jupyterhub.user.eopf.eodc.eu/hub/login?next=%2Fhub%2Fspawn%3Fnext%3D%252Fhub%252Fuser-redirect%252Fgit-pull%253Frepo%253Dhttps%253A%252F%252Fgithub.com%252Feopf-toolkit%252Feopf-101%2526branch%253Dmain%2526urlpath%253Dlab%252Ftree%252Feopf-101%252F02_about_eopf_zarr%252F23_S1_basic_operations.ipynb%23fancy-forms-config=%7B%22profile%22%3A%22choose-your-environment%22%2C%22image%22%3A%22unlisted_choice%22%2C%22image%3Aunlisted_choice%22%3A%224zm3809f.c1.de1.container-registry.ovh.net%2Feopf-toolkit-python%2Feopf-toolkit-python%3Alatest%22%2C%22autoStart%22%3A%22true%22%7D" target="_blank">
  <button style="background-color:#0072ce; color:white; padding:0.6em 1.2em; font-size:1rem; border:none; border-radius:6px; margin-top:1em;">
    🚀 Launch this notebook in JupyterLab
  </button>
</a>

### Introduction

In this notebook, we will explore additional functionalities that can be applied to Sentinel-1 GRD data. Continuing to access the data through the EOPF Zarr sample services, we will learn how to visualize and carry out basic SAR operations, such as georeferencing and backscatter calibration, using the `.zarr` format.

### What we will learn

- 🗺️ How to georeference a Sentinel-1 GRD image using the correct latitude and longitude coordinates?
- ✳ How to calibrate the amplitude over a Sentinel-1 GRD product?
- ✨ How to calibrate the intensity with dB over a Sentinel-1 GRD product?

### Prerequisites

This tutorial uses a re-processed sample dataset from the [EOPF Sentinel Zarr Samples Service STAC API](https://stac.browser.user.eopf.eodc.eu/?.language=en):
- `S1A_IW_GRDH_1SDV_20170508T164830_20170508T164855_016493_01B54C_8604`

Additionally, it is useful to install the extra dependencies from this environment. To do it we just need to run the following command:
```bash
uv sync --all-extras
```



<hr>

#### Import libraries

In [ ]:
import xarray as xr # Basic package to deal with data arrays
import xarray_sentinel # Handles most basic SAR operations
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

<hr>

## Georeferencing GRD product

As seen in the previous [chapter](22_zarr_structure_S1GRD.ipynb), plotting a **Sentinel-1 GRD** item displays an image with `azimuth_time` and `ground_range`coordinates. While these are indeed valid coordinates for **Sentinel-1** products, they are only meaningful within the context of image acquisition. <br>

To compare them with other geospatial datasets or to examine their actual location on Earth, the data needs to be georeferenced using geographic coordinates, such as **latitude** and **longitude**.

### Opening the product

As usual, the first step is to open the Sentinel-1 GRD product we are going to work with. Using the well known functions `open_datatree()` and `open_dataset()`, we open the `.zarr`product.

In [ ]:
url = "https://objectstore.eodc.eu:2222/e05ab01a9d56408d82ac32d69a5aae2a:sample-data/tutorial_data/cpm_b716/S1A_IW_GRDH_1SDV_20170508T164830_20170508T164855_016493_01B54C_8604.zarr"
dt = xr.open_datatree(url, engine='zarr', chunks={})

We can access `.groups` and explore what is inside the polarisation group, as it corresponds to the information we will be working with.<br>
This is going to help us later opening specific subgroups, such as `measurements`subgroup.

In [ ]:
dt.S01SIWGRD_20170508T164830_0025_A094_8604_01B54C_VH.groups

In [ ]:
measurements = dt["S01SIWGRD_20170508T164830_0025_A094_8604_01B54C_VH/measurements"].to_dataset()

As the `measurements.grd`array is large to be directly plotted, we need to slice it. Applying the `.isel()`method is useful in this case.<br>
We observe that the `grd` product coordinates `azimuth_time` and `ground_range` are not stored in geographical coordinates.

In [ ]:
grd = measurements.grd.isel(
    azimuth_time=slice(None, None, 10), ground_range=slice(None, None, 10)
)
grd.plot(vmax=300)
plt.show()

### Setting the GCP

In order to **georeference** the image with the correspondent latitude and longitude coordinates, we need to access and use the ground control points stored into the `.zarr` structure.<br>

These are well known points that contain both the latitude and longitude coordinates but also the azimuth time and ground range coordinates, making it possible to georeference the `grd` image.

In [ ]:
ground_control_point = dt["S01SIWGRD_20170508T164830_0025_A094_8604_01B54C_VH/conditions/gcp"].to_dataset()
ground_control_point

Since we previously downsampled the `grd` product, the coordinate grid (azimuth time and ground range) was processed accordingly. To ensure that the ground control points (latitude and longitude arrays) align properly with this modified grid, we need to apply the same downsample interpolation to them.

We can achieve this using the `.interp_like()` method from `xarray`. This function interpolates the ground control point data to match the dimensions and coordinates of the `grd` product, specifically over the current `azimuth_time` and `ground_range`.

In [ ]:
gcp = ground_control_point.interp_like(grd)

### Georeferencing the product

The final step is to assign the corresponding decimated latitude and longitude values (interpolated from the ground control points) to the `grd` product. This can be done using the `.assign_coords()` method.

After assigning the coordinates, the `grd` dataset will include `latitude` and `longitude` as new entries in its coordinate system. When plotting the grd image, you can then specify `longitude` as the x axis and `latitude` as the y axis. This will display a properly georeferenced image, allowing it to be compared directly with other spatial datasets.

In [ ]:
grd = grd.assign_coords({"latitude": gcp.latitude, "longitude": gcp.longitude})
grd

In [ ]:
grd.plot(x="longitude", y="latitude", vmax=300)
plt.show()

## Calibrating amplitude and intensity

In **Sentinel-1 GRD** products, only the amplitude of the signal is stored, while the phase information is discarded, unlike Single Look Complex (SLC) products, which preserve both. Therefore, it is essential to calibrate the amplitude using the raw Digital Number (DN) values and the additional metadata provided in the `calibration` subgroup.

The relationship between intensity and amplitude is given by:
$$
\text{Intensity} = |\text{Amplitude}|^2
$$


In [ ]:
calibration = dt["/S01SIWGRD_20170508T164830_0025_A094_8604_01B54C_VH/quality/calibration"].to_dataset()

### Amplitude

Since amplitude represents the strength of the radar signal that is backscattered and received by the sensor, amplitude calibration is essential to convert raw digital numbers (DN) into physically meaningful amplitude values. This calibration involves applying the `beta_nought` coefficient, which represents the normalized radar backscatter without accounting for the incidence angle.

The result is a calibrated amplitude image, where pixel values approximate the physical radar backscatter amplitude. To perform this calibration, we use the `.calibrate_amplitude()` method from the `xarray_sentinel` library.

In [ ]:
calibrate_amplitude = xarray_sentinel.calibrate_amplitude(
    grd, calibration.beta_nought
)
calibrate_amplitude.plot(vmax=0.8)
plt.show()

### Intensity

Similar to the amplitude calibration process, intensity calibration can be performed using the `.calibrate_intensity()` method from the `xarray_sentinel` library. This method converts the raw Digital Number (DN) values into radar backscatter intensity. The output is a physically meaningful radar brightness image, usually represented in decibels (dB).

In [ ]:
calibrate_intensity = xarray_sentinel.calibrate_intensity(
    grd, calibration.beta_nought, as_db=True
)
calibrate_intensity.plot(vmin=-20, vmax=5)
plt.show()

### Georeferenced intensity

Because we georeferenced the grd image in the `measurements.grd` dataset before performing the calibrations, both `calibrated_amplitude` and `calibrated_intensity` now include not only `azimuth_time` and `ground_range` as coordinates but also `latitude` and `longitude`.

This allows us to create the same calibrated intensity plot as before, but now georeferenced using geographic coordinates (longitude on the x axis and latitude on the y axis).

In [ ]:
calibrate_intensity

In [ ]:
calibrate_intensity.plot(x="longitude", y="latitude", vmin=-20, vmax=5)
plt.show()

<hr>

## 💪 Now it is your turn

The following exercises will help you to better understand the calibration processes studied before.

### Task 1: Reproduce this workflow on a different area
Using what you’ve already learned about the STAC catalog and Sentinel-1 GRD products, repeat this workflow with a different Sentinel-1 GRD scene. Try to use a scene from the area where you live, just like you did on the previous chapter. Then, observe how the geographical coordinates of the georeferenced image differ between products.

### Task 2: Explore intensity and amplitude values
Discover what happens when the maximum and minimum value on x and y axis change. These will create new results, especially when considering the plots on amplitude and intensity calibration.

### Task 3: Compare the intensity values with other datasets
Try to calibrate and plot some grd products from other datasets and see how the intensity values change over different areas, textures and surfaces.


## Conclusion

During this tutorial we've learnt how to compute amplitude and intensity calibration on Sentinel-1 GRD data and how to georeference `measurements.grd` variable into geographical coordinates. For most of the operations we used `xarray_sentinel` methods.

## What's next?

Now that you have been introduced to the `.zarr` encoding format, learned its core concepts, and understood the basics of how to explore it, you are prepared for the next step. In the following [chapter](./24_zarr_struct_S2L2A.ipynb) we will introduce you to the [Sentinel-2 L-2A](https://stac.browser.user.eopf.eodc.eu/collections/sentinel-2-l2a) `.zarr` structure. As we go along, we are more and more transition from theory to practice, providing you with hands-on tutorials working with EOPF `.zarr` products.